In [ ]:
# Importing necessary libraries
# NOTE: Only works on local computer 
import mysql.connector
import time
import random
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from decimal import Decimal
# Database connection parameters
host = "127.0.0.1"
port = 3306
user = "root"
password = "Mayhemjc22"

try:
    # Connect to the mySQL database
    connection = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database="test"
    )
    
    if connection.is_connected():
        print("Connected to the database")
    
    # Connects Google Drive CSV using Google Drive authentication
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Authenticate using a local webserver
    drive = GoogleDrive(gauth)

    # Defines the CSV file ID
    csv_file_id = '1wZxQe8Sj9FtN7munorspl45K-xAGlpwKOs3R7lvLqkg'
    
    # Perform database operations here
    cursor = connection.cursor()
    
    #Data to be put into the CSV 
    test_data = []
    
    #Header Row for CSV
    header_row = ["DOY", "MINUTE", "Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "SensorSum"]
    test_data.append(header_row)
    
    #Assign Starting Variables
    #Starts Day 1, Minute 1
    DOY = 1
    MINUTE = 1
    
    # Select Data to fetch from mySQL database
    query = f"SELECT * FROM Irradiance WHERE DOY = {DOY} AND MINUTE = {MINUTE}"
    
    #Try to execute query and get the data
    try:
        while True:
            cursor.execute(query)
            result = cursor.fetchall()
            
            #Filters into CSV file format
            result = [[str(item) if isinstance(item, Decimal) else item for item in row] for row in result]
            result = [[float(item.strip("'")) if isinstance(item, str) else item for item in row] for row in result]
            
            test_data.extend(result)
            if result:
                csv_rows = ["\n".join(",".join(map(str, row)) for row in test_data)]
                csv_string = "\n".join(csv_rows)
                file = drive.CreateFile({'id': csv_file_id})
                file.SetContentString(csv_string)
                file.Upload()
            else:
                cursor.close()
                # Close the connection
                connection.close()
                print("Connection closed")
            prev_min = MINUTE
            MINUTE = MINUTE + 60
            if(MINUTE <= 1441):
                query = f"SELECT * FROM Irradiance WHERE DOY = {DOY} AND (MINUTE > {prev_min} AND MINUTE <= {MINUTE})"
            else:
                DOY = DOY + 1
                prev_min = 0
                MINUTE = 1
                query = f"SELECT * FROM Irradiance WHERE DOY = {DOY} AND (MINUTE > {prev_min} AND MINUTE <= {MINUTE})"            
            time.sleep(1)
    # Error Handling
    except mysql.connector.Error as err:
        print("Error:", err)
    #When closing the program, stop the connection and the simulation
    finally:
        cursor.close()
        # Close the connection
        connection.close()
        print("Connection closed")
# Error Handling
except mysql.connector.Error as err:
    print("Error:", err.msg)

Connected to the database
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=204190089206-2p35q64g39bmhrdjc1kdlnfgie8b52q4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
